In [1]:
import tensorflow as tf
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
import numpy as np

In [2]:
pre_trained_model = InceptionV3(
    input_shape=(300, 300, 3), include_top=False, weights='imagenet'
)
# local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
    
# pre_trained_model.summary()

In [3]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 17, 17, 768)


In [4]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation="sigmoid")(x)

model = Model(pre_trained_model.input, x)

model.compile(
    optimizer=RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"]
)

In [5]:
from pathlib import Path

train_dir = Path.cwd() / 'cats-v-dogs' / 'training'
val_dir = Path. cwd() / 'cats-v-dogs' / 'testing'

cat_train = train_dir / 'cats'
dog_train = train_dir / 'dogs'

cat_val = val_dir / 'cats'
dog_val = val_dir / 'dogs'

class_names = np.array([item.name for item in train_dir.glob('*')])
print(f"There is 2 classes: {class_names[0]} and {class_names[1]}")

There is 2 classes: cats and dogs


In [6]:
SEED = 42
BATCH_SIZE = 150
IMG_HEIGHT = 300
IMG_WIDTH = 300

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255.0,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
#     shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode="binary",
)

val_generator = val_datagen.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=val_dir,
    shuffle=False,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode="binary",
)

train_ds = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=tf.TensorSpec(shape=(None, 300, 300, 3), dtype=tf.int32),
)

val_ds = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=tf.TensorSpec(shape=(None, 300, 300, 3), dtype=tf.int32),
)

Found 22283 images belonging to 2 classes.
Found 2473 images belonging to 2 classes.


In [7]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=100,
    epochs=20,
    validation_steps=50,
    verbose=2,
)

Epoch 1/20


ValueError: in user code:

    c:\users\josephdavis\desktop\tensorflow notebooks\env\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    c:\users\josephdavis\desktop\tensorflow notebooks\env\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\josephdavis\desktop\tensorflow notebooks\env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\josephdavis\desktop\tensorflow notebooks\env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\josephdavis\desktop\tensorflow notebooks\env\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replic